In [4]:
import tensorflow as tf

## 1. 索引与切片
### 1.1 索引

In [5]:
# 创建4张32*32大小的彩色图片数据
x = tf.random.normal([4, 32, 32, 3])

# 使用基本索引的方式
a = x[1][2][10]

# 使用tensorflow支持的方式
b = x[1, 2, 10]

a,b

(<tf.Tensor: id=39, shape=(3,), dtype=float32, numpy=array([-0.51165336, -1.8729898 , -0.13762352], dtype=float32)>,
 <tf.Tensor: id=43, shape=(3,), dtype=float32, numpy=array([-0.51165336, -1.8729898 , -0.13762352], dtype=float32)>)

### 1.2 切片

In [6]:
# start:end:step
c = x[0:3:2, 10:20, 10:, :2]

In [11]:
# 读取1-2张图片的G/B通道
d = x[1:2, ..., 1:]

# 读取所有照片的G/B通道
e = x[..., 1:]

# 读取1-2张图片的所有数据
f = x[0:2, ...]

d.shape, e.shape, f.shape

(TensorShape([1, 4, 4, 2]),
 TensorShape([2, 4, 4, 2]),
 TensorShape([2, 4, 4, 3]))

## 2. 维度变换
### 2.1 Reshape

In [13]:
# 改变维度
x = tf.range(96)
x = tf.reshape(x, [2, 4, 4, 3])

注意，在改变维度时，我们需要了解数据的写入顺序，如果写入顺序错误，则改变维度后可能得到错误的结果。

比如正确的特征写入顺序为a-b-c-d，这样我们得到一个一维的向量，将其按照[2, 4, 4, 3]reshpae后，得到正确的结果。

但是如果写入顺序为a-c-b-d，虽然也能得到一个一维向量，但是按照[2, 4, 4, 3]reshape后，得到的结果并不正确。

In [14]:
# 查看维度数量和形状
x.ndim, x.shape

(4, TensorShape([2, 4, 4, 3]))

In [15]:
# 如果有一个维度的长度不知道，或者懒得计算
# 可以用-1，让计算机自动计算该维度的长度
x = tf.reshape(x, [2, 3, -1])
x.shape

TensorShape([2, 3, 16])

上面计算机自动将最后一个维度的长度计算出来，为16

### 2.2 增删维度
无论是增加一个维度还是删除一个维度，增加和删除的维度长度必须为1，因此并不改变数据的存储，只是改变数据的理解方式

In [16]:
x.shape

TensorShape([2, 3, 16])

In [17]:
# 增加维度，axis = 2说明在第二个维度后面增加一个维度
x = tf.expand_dims(x, axis = 2)
print(x.shape)

# 删除刚才增加的维度，axis = 2说明删除第二个维度后面的维度
x = tf.squeeze(x, axis = 2)
print(x.shape)

(2, 3, 1, 16)
(2, 3, 16)


In [18]:
# 增加维度，axis = -3说明从后往前数，在第三个位置增加一个维度
x = tf.expand_dims(x, axis = -3)
print(x.shape)

# 删除刚才增加的维度，axis = -3说明从后往前数，删除第三个位置的维度
x = tf.squeeze(x, axis = -3)
print(x.shape)

(2, 1, 3, 16)
(2, 3, 16)


如果squeeze不指定维度，将删除所有长度为1的维度

### 2.3 交换维度

In [19]:
x1 = tf.random.normal([2, 23, 23, 3])
x2 = tf.transpose(x1, perm = [0, 3, 1, 2])
x1.shape, x2.shape

(TensorShape([2, 23, 23, 3]), TensorShape([2, 3, 23, 23]))

x1是原始的维度排列方式，如果将原始的维度排列方式按照顺序编号，即0，1，2，3。我们在tf.transpose方法中，参数perm表示将原始维度按照perm的编号重新进行排列，上面的例子是将第4个维度放在第二的位置。

### 2.4 数据复制

In [20]:
x = tf.range(4)
x = tf.reshape(x, [2,2])

# 在列方向上复制一次数据
x1 = tf.tile(x, multiples = [1,2])

# 在行方向上再复制一次数据
x2 = tf.tile(x1, multiples = [2,1])

# 当然也可以进行多次复制
x3 = tf.tile(x, [2,3])
x, x1, x2, x3

(<tf.Tensor: id=135, shape=(2, 2), dtype=int32, numpy=
 array([[0, 1],
        [2, 3]])>,
 <tf.Tensor: id=137, shape=(2, 4), dtype=int32, numpy=
 array([[0, 1, 0, 1],
        [2, 3, 2, 3]])>,
 <tf.Tensor: id=139, shape=(4, 4), dtype=int32, numpy=
 array([[0, 1, 0, 1],
        [2, 3, 2, 3],
        [0, 1, 0, 1],
        [2, 3, 2, 3]])>,
 <tf.Tensor: id=141, shape=(4, 6), dtype=int32, numpy=
 array([[0, 1, 0, 1, 0, 1],
        [2, 3, 2, 3, 2, 3],
        [0, 1, 0, 1, 0, 1],
        [2, 3, 2, 3, 2, 3]])>)

### 2.5 Broadcasting

In [21]:
a = tf.range(12)
a = tf.reshape(a, [3, 4])
b = tf.range(4)
a.shape, b.shape

(TensorShape([3, 4]), TensorShape([4]))

现在我们有了一个3*4的矩阵和一个长度为4的向量，当两个矩阵的形状不一样时，应该是不能相加的，但是在broadcasting机制下，相加成功了

In [22]:
a + b

<tf.Tensor: id=152, shape=(3, 4), dtype=int32, numpy=
array([[ 0,  2,  4,  6],
       [ 4,  6,  8, 10],
       [ 8, 10, 12, 14]])>

这是为什么呢？对于b来说，它实际上是一个[1, 4]的矩阵，broadcasting可以将长度维1的维度自动扩展到与其运算的另一个矩阵的长度，来完成运算，也就是将b扩展成了[3, 4]的矩阵，扩展出来的长度中保存的是原始数据的复制

In [23]:
# 通过tf.broadcast_to，可以将矩阵b扩展到与a相同的形状
B = tf.broadcast_to(b, a.shape)
B

<tf.Tensor: id=154, shape=(3, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])>

In [24]:
# 那么对于[1, 4]和[3, 1]的两个矩阵，是否可以broadcasting呢？事实上也是可以的
a = tf.range(4)
b = tf.range(3)
b = tf.reshape(b, [3, 1])
a + b

<tf.Tensor: id=165, shape=(3, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5]])>

## 3. 数学运算
### 加减乘除
- 加  +
- 减  - 
- 乘  *
- 除  /
- 整除  //
- 余除  %
- 乘方  **
- 平方  tf.square()
- 平方根  tf.sqrt()
- 指数  tf.pow(a, x)或**
- 自然指数  tf.exp(x)
- 对数  tf.math.log(x)
- 矩阵相乘  @或者tf.matmul(a, b)

In [25]:
a = tf.reshape(tf.range(4), [2, 2])
b = tf.reshape(tf.range(6), [2, 3])
a @ b, tf.matmul(a, b)

(<tf.Tensor: id=178, shape=(2, 3), dtype=int32, numpy=
 array([[ 3,  4,  5],
        [ 9, 14, 19]])>,
 <tf.Tensor: id=179, shape=(2, 3), dtype=int32, numpy=
 array([[ 3,  4,  5],
        [ 9, 14, 19]])>)